# use CSV upload web browser

In [1]:
import networkx as nx
import json 
import os

# these are the two functions one needs to create a JSON file to upload and create the project in the backend 
import nx2json as nx2j
import uploaderGraph as uG

# CREATE A NX.GRAPH OBJECT per layout

### nx.Graph

In [2]:
G = nx.circular_ladder_graph(200)
print("Number of nodes: ", len(G.nodes()))
print("Number of Links: ", len(G.edges()))

# ===============================================
# GRAPH NAME AND DESCRIPTION - a string each
# ===============================================

G.graph['projectname'] = "CircularLadder-csv"
G.graph['info'] = "A toy graph for testing purposes. Number of nodes: "+str(len(G.nodes()))+", Links: "+ str(len(G.edges()))+"."

Number of nodes:  400
Number of Links:  600


In [3]:
#nx.draw(G)

### create node anntotations

In [4]:
import random 
def generate_random_words(num):
    words = ["alpha", "beta", "gamma", "delta", "epsilon", "zeta", "theta", "lambda", "mu", "nu"]
    return random.sample(words, num)

# Create a list to hold annotations in JSON format
l_annotations_json = []

# Process each node in the graph
for g in G.nodes():
    # Generate random annotations
    annotations = {
        "annot1": generate_random_words(random.randint(3, 4)),
        "annot2": generate_random_words(random.randint(1, 2)),
        "annot3": generate_random_words(random.randint(2, 3))
    }
    
    l_annotations_json.append(annotations)

# Create a dictionary mapping nodes to their annotations
d_annotations = dict(zip(G.nodes(), l_annotations_json))

# Set the node attributes in the graph
nx.set_node_attributes(G, d_annotations, name="annotation")

In [5]:
# set node names (optional)
for n in G.nodes():
    G.nodes[n]['name'] = "Nodename_"+str(n)

### create node positions and set as "pos" Graph attribute 
here are 3 different layouts, which all are stored in unique nx.Graph-objects (G_rgba, G_hex, ....)

In [6]:
# First layout (i.e. Graph 1)
G_rgba = G.copy()
G_rgba.graph["layoutname"] ='layout1-spring'
posG3D_1_pre = nx.spring_layout(G_rgba, dim=3, k=0.1, iterations=50)
posG3D_1 = {key: value.tolist() for key, value in posG3D_1_pre.items()}
nx.set_node_attributes(G_rgba, posG3D_1, name="pos")

#### node and link colors 

In [7]:
# 3 Formats of colors values are supported: hex, rgba, hex8

d_nodecolors_rgba = dict(zip(G_rgba.nodes(),[(0,0,255,200)]*len(G_rgba.nodes())))
nx.set_node_attributes(G_rgba, d_nodecolors_rgba, name="nodecolor")
l_linkcolors_rgba = (0,0,255,200)
nx.set_edge_attributes(G_rgba, l_linkcolors_rgba, name="linkcolor")

# make CSV tables and export

In [8]:
import pandas as pd

# x y z 
df = pd.DataFrame(posG3D_1.values(), columns=['x', 'y', 'z'])
df.to_csv(G_rgba.graph['layoutname']+'_node_table.csv', index=False, header = False)

# r g b a
df_colors = pd.DataFrame(d_nodecolors_rgba.values(), columns=['r', 'g', 'b', 'a'])
df_colors.to_csv(G_rgba.graph['layoutname']+'_node_color.csv', index=False, header = False)

# links
df_links = pd.DataFrame([(u,v) for u,v in G_rgba.edges()], columns=['source', 'target'])
df_links.to_csv(G_rgba.graph['layoutname']+'_link_table.csv', index=False, header = False)

# link colors
df_linkcolors = pd.DataFrame([l_linkcolors_rgba]*len(G_rgba.edges()), columns=['r', 'g', 'b', 'a'])
df_linkcolors.to_csv(G_rgba.graph['layoutname']+'_link_color.csv', index=False, header = False)

# THE FOLLOWING SECTION IS JUST FOR BACKGROUND INFO - no need to run
This is how the graph is storing all information given nx.Graph(s).

In [ ]:
'''

{
----------------------------------------
THIS IS THE GENERAL GRAPH INFO SECTION
----------------------------------------
  "directed": false,
  "multigraph": false,
  "projectname": "Testgraph",
  "info": "A toy graph for testing purposes. Number of nodes: 10, Links: 43.",
  "graphlayouts": [
      "layout1-spring",
      "layout2-spring",
      "layout3-spring",
      "layout4-clusters"
  ],
  "annotationTypes": true,
  "nodes": [
   ----------------------------------------
   contains all nodes of the project
   ----------------------------------------
      {
          "id": 0,
          "name": nodename-x,
          "annotation": 
                {
                    "annot1": [
                        "lambda",
                        "alpha",
                        "zeta",
                        "theta"
                    ],
                    "annot2": [
                        "delta",
                        "nu"
                    ],
                    "annot3": [
                        "mu",
                        "gamma"
                    ]
          }
      },....
  ],
  "links": [
   ----------------------------------------
   contains all links of the project
   ----------------------------------------
      {
          "id": 0,
          "source": 0,
          "target": 1
      },
      {
          "id": 1,
          "source": 0,
          "target": 2
      },...
       ],
  "layouts": [
   ----------------------------------------
   contains all layouts of the project
   only contains nodes and links as well as colors specific to the layout
   ----------------------------------------
       {  "layoutname" : "name of first layout",
          "nodes": [
              {
                  "nodecolor": [
                      255,
                      35,
                      0,
                      120
                  ],
                  "pos": [
                      -0.5618057865250979,
                      0.1467411221839164,
                      0.49656801102094605
                  ],
                  "id": 0
               },...
        	],
          "links": [
              {
                  "linkcolor": [
                      0,
                      255,
                      0,
                      100
                  ],
                  "source": 0,
                  "target": 1
              },...
         	],
   	  }, {
          "layoutname" : "name of second layout",
          "nodes": [
              {
                  "nodecolor": "#0000ffaa",
                  "pos": [
                      -0.35948900932978317,
                      0.6255258442839948,
                      -0.04209289102217994
                  ],
                  "cluster": "cluster group 1",
                  "id": 0
               },... 
],
          "links": [
              {
                  "linkcolor": "#0000ff",
                  "source": 0,
                  "target": 1
              },
],
  	   }, { . . .  
 	},
}

'''

## CREATE A JSON FILE WITH THE ABOVE STRUCTURE to then create a project

In [28]:
import networkx as nx
import json 
import os

# these are the two functions one needs to create a JSON file to upload and then create the project in the backend 
import nx2json as nx2j 
import uploaderGraph as uG

In [14]:
# ----------------------------------------
# CREATE Json file
# ----------------------------------------
merged_graphs = nx2j.make_json(Graphs)
path = "temp_files/"

# save the merged graph in a json file
with open(path+Graphs[0].graph['projectname']+'.json', 'w') as fp:
    json.dump(merged_graphs, fp, indent=4)

In [ ]:
# ----------------------------------------
# READ Json file
# ----------------------------------------
filename = 'myfile.json'
currentwd = '.../DataDiVR_Webapp/temp_files/' # modify file location here
path = os.path.join(currentwd, filename)

# open the json file
with open(path, 'r') as f:
     G_merged = json.load(f)

In [ ]:
## ----------------------------------------
# CREATE A PROJECT for the VR Platform 
# ----------------------------------------
#the actual "upload step" to create a project with the required VR platform files 

uG.upload_filesJSON(G_merged)